<a href="https://colab.research.google.com/github/martings/My_own_single_cell_toolkit/blob/main/Seurat_EDA_e_integracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#En colab tarda unos 25 minutos en instalarse el Seurat
install.packages('Seurat')
#install.packages('tidyverse')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘plyr’, ‘tensor’, ‘BH’, ‘sitmo’, ‘globals’, ‘listenv’, ‘parallelly’, ‘zoo’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’, ‘RcppTOML’, ‘here’, ‘gplots’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘sp’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘spatstat.data’, ‘spatstat.random’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘goftest’, ‘abind’, ‘deldir’, ‘polyclip’, ‘FNN’, ‘dqrng’, ‘cowplot’, ‘fitdistrplus’, ‘future’, ‘future.apply’, ‘ggrepel’, ‘ggridges’, ‘ica’, ‘igraph’, ‘irlba’, ‘leiden’, ‘lmtest’, ‘matrixStats’, ‘miniUI’, ‘patchwork’, ‘pbapply’, ‘plotly’, ‘png’, ‘progressr’, ‘RANN’, ‘Rcpp’, ‘RcppAnnoy’, ‘reticulate’, ‘ROCR’, ‘Rtsne’, ‘scattermore’, ‘sctransform’, ‘SeuratObject’, ‘shiny’, ‘spatstat.explore’, ‘spatstat.geom’, ‘uwot’, ‘RcppEigen’, ‘RcppProgress’


The legacy packages maptools, rgdal, and rgeos, underpinning this package
will re

In [2]:
library(Seurat)
suppressMessages({
    library(tidyverse)
})

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject



In [8]:
#Analizo uno de los archivos a integrar para ver que pinta tienen
AdultMG_Visvader<-readRDS("/content/SC_MG_Data/Visvader.10x.combined.BackUP.dim30.rds")

In [ ]:
pbmc <- SCTransform(AdultMG_Visvader)
rm(AdultMG_Visvader)
pbmc <- RunPCA(pbmc)
pbmc <- RunUMAP(pbmc, dims = 1:20)
pbmc <- FindNeighbors(pbmc, dims = 1:20)
pbmc <- FindClusters(pbmc, resolution = 0.1 )
DimPlot(pbmc, reduction = "umap")

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 13224 by 22287

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Found 26 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 13224 genes



  |======================================================================| 100%


Computing corrected count matrix for 13224 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 2.256748 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix



In [7]:
#Cargo los dataset de entrada para realizar la integracion

Khaled_total<-readRDS("/content/SC_MG_Data/Khaled.10x.combined.estricto.features.BackUP.dim30.rds")
AdultMG_Spike<-readRDS("/content/SC_MG_Data/AdultMG_combined_BackUp.rds")
AdultMG_Visvader<-readRDS("/content/SC_MG_Data/Visvader.10x.combined.BackUP.dim30.rds")

#Armo una lista con todos los objetos a integrar
seurat_object_list<-c("Khaled_total","AdultMG_Spike","AdultMG_Visvader")
pbmc.list <- list()
for (i in seurat_object_list) {
  pbmc.list[i] <- list(get(i))
}
#Borro los archivos originales de memoria RAM
rm(Khaled_total)
rm(AdultMG_Spike)
rm(AdultMG_Visvader)

#Transformo y normalizo los datos a integrar
for (i in names(pbmc.list)) {
  pbmc.list[[i]] <- SCTransform(pbmc.list[[i]], verbose = T)
}

#Aca busco los features (genes) que van a ser usados para realizar la integracion. 
#Busca los genes que tienen buena expresion y buen peso estadistico entre los tres data set como para usarlos de featues principales.
#El numero de features, va a mejorar la calidad/resolucion de la integracion, pero va a requerir mas RAM para realizar la integracion
pbmc.features <- SelectIntegrationFeatures(object.list = pbmc.list, nfeatures = 3000)

#Realiza la normalizacion SCT a los datos a integrar
pbmc.list <- PrepSCTIntegration(object.list = pbmc.list, anchor.features = pbmc.features)

#Se puede usar un dataset de referencia, de mapa principal al cual el resto de los dataset van a tratar de amolgarse. 
#Es util por si queremos conservar las separaciones de alguno de los dataset antes que otros.
#El programa puede correr sin referencia puntual o mismo puede hacerlo con varios. 
reference_dataset <- which(names(pbmc.list) == "Khaled_total")

#""Centra"" los datos de los tres dataset normalizados o tambien se puede pensar que los limita a un conjunto de features que contempla lo mejor posible los 3.
pbmc.anchors <- FindIntegrationAnchors(object.list = pbmc.list, normalization.method = "SCT", 
                                       anchor.features = pbmc.features, reference = reference_dataset)
#Borro por cuestiones de RAM la lista original
rm(pbmc.list)
#Con todos los datos centados y normalizados, los integro todos a un nuevo dataset
pbmc.integrated <- IntegrateData(anchorset = pbmc.anchors, normalization.method = "SCT")
#Es conveniente guardar el archivo crudo para tener a disposicion si 
saveRDS(pbmc.integrated, "pbmc.integrated.backup.rds")
#Siempre es bueno ahorrar RAM
rm(pbmc.anchors)

#
pbmc.integrated <- SCTransform(pbmc.integrated,nfeatures = 3000 )
pbmc.integrated <- RunPCA(pbmc.integrated)
pbmc.integrated <- RunUMAP(pbmc.integrated, dims = 1:20)
pbmc.integrated <- FindNeighbors(pbmc.integrated, dims = 1:20)
pbmc.integrated <- FindClusters(pbmc.integrated, resolution = 0.1 )
DimPlot(pbmc.integrated, reduction = "umap", group.by = c("orig.ident", "Cell_Type"))

saveRDS(pbmc.integrated, "pbmc.integrated.post.rds")

ERROR: ignored